Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class



In [3]:
import pandas as pd

def sort_film(row):
    if row['rating'] <= 2:
        return "низкий рейтинг"
    if row['rating'] > 2 and row['rating'] <= 4:
        return "средний рейтинг"
    if row['rating'] > 4:
        return "высокий рейтинг"


data = pd.read_csv('ratings.csv')
data['class'] = data.apply(sort_film, axis=1)
data.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,высокий рейтинг
4,1,50,5.0,964982931,высокий рейтинг


Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.



In [8]:
import pandas as pd
import re

regg  = '(москва | тула | ярославль)'
regg1  = '(петербург | псков | мурманск)'
regg2  = '(владивосток | сахалин | хабаровск)'

def prov_region(row):
    if len(re.findall(regg, row['keyword'])) > 0:
        return 'Центр'
    if len(re.findall(regg1, row['keyword'])) > 0:
        return 'Северо-Запад'
    if len(re.findall(regg2, row['keyword'])) > 0:
        return 'Дальний Восток'
    return 'undefined'

data = pd.read_csv('keywords.csv')
data['region'] = data.apply(prov_region, axis=1)
data.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


Проверка

In [9]:
data['region'].value_counts().head()

undefined         99686
Центр               198
Северо-Запад         70
Дальний Восток       46
Name: region, dtype: int64

Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга



In [10]:
import pandas as pd
import re
data = pd.read_csv('ratings.csv')
data1 = pd.read_csv('movies.csv')
year = list(range(1950,2011))
data2 = data.merge(data1, how='left').filter(items = ['rating', 'movieId', 'title'])
regg = '(?<=\()[\d]{4}(?=\))'

def production_year(row):
    prov = re.findall(regg, row['title'])
    if len(prov) > 0:
        for i in year:
            if i == int(prov[0]):
                return prov[0]
        else:
            return '1900'

data2['year'] = data2.apply(production_year, axis=1)
data2.head(50)
data2.groupby('year').mean()[['rating']].sort_values(by='rating', ascending=False).head(25)

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1964,3.940160
1972,3.939891
1974,3.935622
1967,3.922572
1975,3.879121


Вывод - действительно старые фильмы обладают более высоким рейтингом.

ВОПРОС! Можно ли группировать не просто по годам, а по диапазону? Например 1950-1959, 1960-1969 и т .д.?